In [1]:
! apt-get install git
! git clone https://github.com/joaorossi15/requirements-to-us.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'requirements-to-us'...
remote: Enumerating objects: 15975, done.
remote: Counting objects: 100% (5076/5076), done.
remote: Compressing objects: 100% (3734/3734), done.
remote: Total 15975 (delta 1342), reused 5054 (delta 1323), pack-reused 10899 (from 1)
Receiving objects: 100% (15975/15975), 200.77 MiB | 16.51 MiB/s, done.
Resolving deltas: 100% (3490/3490), done.


In [2]:
! pip install datasets
! pip install optimum
! pip install auto-gptq
! pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
 

In [3]:
import transformers
import peft
import pandas as pd
import datasets
import torch
import gc
import re
import os

os.environ["WANDB_DISABLED"] = "true"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device
gc.collect()

30

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
def load_model(model_name: str):
    m = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='cuda',
        trust_remote_code=False,
        revision='main',
    )

    return m.to(device)


model_name = "mistralai/Mistral-7B-Instruct-v0.3"
m = load_model(model_name=model_name)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
def construct_instruction() -> str:
    transparency = "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication."
    n_malef = "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others"
    resp = "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity"
    privacy = "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result"
    benefiecnce = "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole"
    f_and_a = "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us"
    sus = "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas"
    dig = "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected"
    j_and_f = "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality."
    trust = "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective."
    instruction = f'You are an ethical requirements engineer translating requirements to ethical user stories based on one of the 10 ethical principles: {transparency}\n {n_malef}\n {resp}\n {privacy}\n {benefiecnce}\n {f_and_a}\n {sus}\n {dig}\n {j_and_f}\n {trust}\n and the TEMPLATE:\n As a <persona> i want to <do something> <so that>\n Now following this template and the ethical principles defined above, choose only one of the ethical principles and transform the requirement below into a description in one line of an ethical user story, substituting <persona> for the one that requires this functionality, <do something> for what the <persona> wants to do and <so that> to the end goal of the functionality'
    instruction = instruction.lower()

    return instruction

def text_to_lower(text: str) -> str:
    return text.lower()

def pad_multiple_spaces(text: str) -> str:
    return re.sub( r"\s+"," ",text)

def apply_in_row(row, instruction):
    return f'''<s>[INST] {instruction} \n {row['text']} \n [/INST] {row['ethical_us']}</s>'''



In [9]:
def load_dataset():
    df = pd.read_csv('./requirements-to-us/data/data_final.csv')
    df = df[['text', 'is_major_principle_here', 'ethical_us']]

    return df

def prepare_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df['text'] = df['text'].apply(text_to_lower)
    df['ethical_us'] = df['ethical_us'].apply(text_to_lower)
    df['text'] = df['text'].apply(pad_multiple_spaces)
    df['ethical_us'] = df['ethical_us'].apply(pad_multiple_spaces)
    instruction = construct_instruction()
    df['data'] = df.apply((lambda row: apply_in_row(row, instruction)), axis=1)

    return df

def tokenize_dataset(model_name: str):
    t = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=True)
    t.pad_token = t.eos_token
    data_collator = transformers.DataCollatorForLanguageModeling(t, mlm=False)

    return data_collator, t

def train_test_split(df: pd.DataFrame) -> pd.DataFrame:
    df = datasets.Dataset.from_pandas(df)
    df = df.train_test_split(test_size=0.2)

    return df


In [10]:
df = load_dataset()
df = prepare_dataset(df)
data_collator, t = tokenize_dataset(model_name)
data = train_test_split(df)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [11]:
def tokenize_function(df):
    # extract text
    text = df["data"]

    #tokenize and truncate text
    t.truncation_side = "left"
    tokenized_inputs = t(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/741 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

In [12]:
def train_model(model, lr, batch_size, num_epochs, tokenized_data, collator):
    model.train() # training state
    model.gradient_checkpointing_enable()
    model = peft.prepare_model_for_kbit_training(model) # turn into qlora

    # lora config
    config = peft.LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=["q_proj"],
        lora_dropout=.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    config.inference_mode = False

    model = peft.get_peft_model(model, config) # model in lora style

    training_args = transformers.TrainingArguments(
        output_dir= "checkpoints_output",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        warmup_steps=2,
        fp16=True,
        optim="paged_adamw_8bit",
        report_to=None
    )

    trainer = transformers.Trainer(
        model=model,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        args=training_args,
        data_collator=collator,
    )

    # train model
    model.config.use_cache = False
    trainer.train()

    # renable warnings
    model.config.use_cache = True

    return model, trainer

final_model, trainer = train_model(m, 1e-4, 4, 3, tokenized_data, data_collator)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.543700,0.243850
2,0.238100,0.227025
3,0.219600,0.221363


In [13]:
prompt_template = lambda comment: f'''[INST] Transform the requirement below into a one line brief description of an ethical user story \n {comment} \n[/INST]'''
comment = "The user need an authentication token to login."
prompt = prompt_template(comment)

final_model.eval()
inputs = t(prompt, return_tensors="pt")
outputs = final_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=52)
print(t.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s>[INST] Transform the requirement below into a one line brief description of an ethical user story 
 The user need an authentication token to login. 
[/INST] Ethical user story: As a user, I want to have an authentication token for secure and authorized login access.

This user story emphasizes the importance of user authentication for security purposes, ensuring that only authorized users can access the system.</s>


In [14]:
final_model.save_pretrained("./requirements-to-us/final-model")

In [15]:
trainer.push_to_hub("joaorossi15/mistral-7B-v03-ai-ethics-translator")

events.out.tfevents.1730938580.f6bbeb0caa12.905.0:   0%|          | 0.00/6.75k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/joaorossi15/checkpoints_output/commit/32485f3241ec466af14f5ee94ce55576849349b3', commit_message='joaorossi15/mistral-7B-v03-ai-ethics-translator', commit_description='', oid='32485f3241ec466af14f5ee94ce55576849349b3', pr_url=None, pr_revision=None, pr_num=None)